## 一、矩阵

矩阵其实就是数的表格，它的加法减法数乘基本没啥好说的，一看就懂。

矩阵更高层次的内涵是：**矩阵是线性操作（线性变换）的一种抽象表达**。

矩阵运算的重点和难点是 **矩阵乘法** 和 **矩阵的逆（除法）**。

其中矩阵的逆，理论价值大于应用价值，数值计算中很少使用矩阵的逆（因为效率低下）。

这里就着重强调矩阵乘法。

## 二、从不同的视角去理解矩阵乘法（Perspectives on matrix multiplication）

### 2.1 矩阵乘法的定义：行 × 列

$$
\begin{pmatrix}
 -14 &   5 & 10 \\
  \color{red}{-5} & -20 & 10 \\
  -6 &  10 &  6
\end{pmatrix} =
\begin{pmatrix}
 2 & -1 & 5 \\
  \color{red}{3} &  \color{red}{4} & \color{red}{4} \\
 -4 & -2 & 0
\end{pmatrix}
\begin{pmatrix}
\color{red}{1}  & 0 & -2 \\
  \color{red}{1} & -5 &  1 \\
 \color{red}{-3} &  0 &  3
\end{pmatrix}
$$
 $\color{red}{-5 = 3 \times 1 + 4 \times 1 + 4 \times -3}$ ($A$ 的第二行 ⋅$B$ 的第二列).

通用公式为
$$
c_{ij} = \sum_{k=1}^n a_{ik} b_{kj}
$$

In [1]:
A = [ 2  -1  5
      3   4  4
     -4  -2  0]
B = [ 1   0  -2
      1  -5   1
     -3   0   3]
C = A * B

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

In [7]:
# 矩阵乘法按定义的实现

function matmul_ijk(A,B)
    m,n = size(A)
    n2,p = size(B)
    if n≠n2 error("A 的列数必须等于 B 的行数！") end
   
    C = zeros(m, p)
    
    for i=1:m, j=1:p, k=1:n  # 三层循环
       C[i,j] += A[i,k]*B[k,j]
    end
    
    return C
end

matmul_ijk(A, B) == A * B

true

这个是最枯燥无聊的定义，通过调整计算的顺序，我们可以得到矩阵乘法更直观的描述。

而其中尤其重要的，就是是下面的 **行视点**和 **列视点**。

### 2.2 列视点：矩阵 × 列向量（matrix × columns）

$$
\begin{pmatrix}
  \color{red}{-14} &  \color{blue}{5}   &  \color{orange}{10} \\
  \color{red}{-5}  &  \color{blue}{-20} &  \color{orange}{10} \\
  \color{red}{-6}  &  \color{blue}{10}  &  \color{orange}{6}
\end{pmatrix} =
\begin{pmatrix}
  2 & -1 & 5 \\
  3 &  4 & 4 \\
 -4 & -2 & 0
\end{pmatrix}
\begin{pmatrix}
  \color{red}{1}  &  \color{blue}{0}  &  \color{orange}{-2} \\
  \color{red}{1}  &  \color{blue}{-5} &  \color{orange}{1}  \\
  \color{red}{-3} &  \color{blue}{0}  &  \color{orange}{3}
\end{pmatrix}
$$

$C$ 的第 $j$ 列，等于 $A$ 乘以 $B$ 的第 $j$ 列

即 `C[:, j] = A * B[:, j]`，这就相当于多个 $Ax=b$.

In [14]:
A * B[:,1]  # A × B的第一列 = C 的第一列

3-element Array{Int64,1}:
 -14
  -5
  -6

In [15]:
[ A*B[:,1]  A*B[:,2]  A*B[:,3] ]  # 结果应该等于 C

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

#### 2.3.2 用途

在线性方程组里，我们将 $Ax$ 解释为对 $矩阵A$ 的列向量的线性组合（是下一节的笔记）。这里的 $C[:, i] = A * B[:, i]$ 同样可以如此理解。

列视点可用于对 $矩阵A$ 的列向量做线性组合，也就是**做初等列变换**！

初等列变换包含下列三种操作：
- **加减**：从某列减掉另一列的k倍（或加上）
- **交换**：互换某两列
- **数乘**：将某列的元素全部乘上某个非零常数。

**$A$ 右乘一个变换矩阵 $B$，可以对 $A$ 实现上述的列变换。**

In [25]:
A

3×3 Array{Int64,2}:
  2  -1  5
  3   4  4
 -4  -2  0

In [28]:
A * [-1 0 0
     0 0 1
     0 1 0]  # 第一列乘上-1，并交换第二列和第三列

3×3 Array{Int64,2}:
 -2  5  -1
 -3  4   4
  4  0  -2

### 2.3 行视点：行向量 × 矩阵（rows × matrix）

$$
\begin{pmatrix}
  \color{red}{-14} &  \color{red}{5}   &  \color{red}{10} \\
  \color{blue}{-5}  &  \color{blue}{-20} &  \color{blue}{10} \\
  \color{orange}{-6}  &  \color{orange}{10}  &  \color{orange}{6}
\end{pmatrix} =
\begin{pmatrix}
 \color{red}{2} & \color{red}{-1} & \color{red}{5} \\
  \color{blue}{3} &  \color{blue}{4} & \color{blue}{4} \\
 \color{orange}{-4} & \color{orange}{-2} & \color{orange}{0}
\end{pmatrix}
\begin{pmatrix}
  1 &  0 & -2 \\
  1 & -5 &  1 \\
 -3 &  0 &  3
\end{pmatrix}
$$

$C$ 的第 $i$ 行，等于 $A$的第 $i$行 乘以 $B$。

即 `C[i, :] = A[i, :] * B`.

In [21]:
A[1,:]'*B  # 应该等于 C 的第一行

1×3 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 -14  5  10

In [24]:
[ A[1,:]'*B 
  A[2,:]'*B
  A[3,:]'*B ]  # 应该等于 C

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

行视点可用于对 $矩阵B$ 的行向量做线性组合，也就是**做初等行变换**！

初等列变换包含下列三种操作（和初等列变换完全类似）：
- **加减**：从某行减掉另一行的k倍（或加上）
- **交换**：互换某两行
- **数乘**：将某行的元素全部乘上某个非零常数。

**$B$ 左乘一个变换矩阵 $A$，可以对 $B$ 实现上述的行变换。**

In [29]:
B

3×3 Array{Int64,2}:
  1   0  -2
  1  -5   1
 -3   0   3

In [33]:
[ 0 1 0
  1 0 0
  3 0 1 ] * B  # 交换第一行和第二行，并在第三行加上（原来的）第一行的三倍

3×3 Array{Int64,2}:
 1  -5   1
 1   0  -2
 0   0  -3

### 2.4 列 × 行

>最重要的是上述的两个视点，这个相对就没那么重要。

$AB=C$ 可以看成下列形式
$$
\begin{pmatrix}
  a_1 & a_2 & ... & a_n
\end{pmatrix}
\begin{pmatrix}
  b_1 \\
  b_2 \\
  ... \\
  b_n
\end{pmatrix}=
\sum_{i=1}^n a_{i} b_{i}
$$

其中 $a_i$ 是 $矩阵A$ 的列向量，$b_i$ 是 $矩阵B$ 的行向量，$a_i b_i$ 得到的是一个 $m × p$ 的矩阵。

从这个视点也可发现些有趣的现象：若给 $a_i$ 乘上非零常数 $k$，给 $b_i$ 乘上 $1 \over k$。$AB=C$ 仍然成立，因为这个改变只会影响上述求和式中的 $a_i b_i$ 项，而该项的结果为 $k a_i \frac{1}{k}b_i = a_i b_i$，没有变化。

**NOTE:变换矩阵E一定是方阵，但是被变换的矩阵A就很随意了。**具体而言， $m × n$ 的矩阵 A，m 和 n 可以随意。但是它的行变换矩阵应该是 $m × m$ 的，列变换矩阵应该是 $n × n$ 的。

### 2.5 子矩阵：分块计算

$
\left[
    \begin{array}{c|c}
        A_1        & A_2 \\
        \hline A_3 & A_4
    \end{array}
\right]
\left[
    \begin{array}{c|c}
        B_1        & B_2 \\
        \hline B_3 & B_4
    \end{array}
\right]=
\left[
    \begin{array}{c|c}
        A_1 B_1+A_2 B_3         & A_1 B_2+A_2 B_4 \\
        \hline A_3 B_1+A_4 B_3  & A_3 B_2+A_4 B_4
    \end{array}
\right]
$

可以看到矩阵分块后，仍然满足矩阵乘法规则。

对 $m × n$ 的矩阵 $A$ 和 $n × p$ 的 矩阵 $B$ 而言，**矩阵的分块计算只要求 $A$ 的列的划分要和 $B$ 的行的划分一致**，这样 $A_i B_i$ 这个乘法才能成立。而A的行的划分，和B的列的划分，是很随意的。

**用途**：分块最重要的应用，应该是分布式计算/并行计算。此外，某些情况下分块可以减少计算量。

In [13]:
# 举例（注意非方阵也可以做分块）

A₁ = [1 2 7
      3 4 6
      5 -7 5]
A₂ = [1 1
      -2 3
      7 3]
A = [A₁ A₂]

B₁ = [3 -4
      2 3
      6 5]
B₂ = [-2 1
      6 5]
B = [B₁  # B₁ 是 2*2
     B₂] # B₂ 是 1*2

@show A*B
@show A₁ * B₁ + A₂ * B₂  # 应该等于 A*B

A * B = [53 43; 75 43; 35 6]
A₁ * B₁ + A₂ * B₂ = [53 43; 75 43; 35 6]


3×2 Array{Int64,2}:
 53  43
 75  43
 35   6

## 三、矩阵乘法的性质

先定义两个常用方阵
1. 对角线元素为1，其他元素全为0的方阵，称为单位矩阵（Identity Matrix，eye），记做 $I$。
1. 对单位矩阵做**单次**初等变换（行/列）所得的方阵，叫做初等矩阵。


1. 矩阵乘法不具备交换性，$AB$ 一般不等于 $BA$. (特殊情况如 $AA^{-1}=I=A^{-1}A$)
1. $IA=AI=A$
1. 对单位矩阵做一些初等行(列)变换得到矩阵 E，将同样的行(列)变换应用到 $A$ 上将得到 $EA$（$AE$）。